# Creating Vector Data from Concatenated Data (2D) after Sampling

For each time series feature of SWAN-SF, we calculate nine statistical features to convert the multivariate time series data (Concatenated) into vector data.

In [4]:
# NewFeatures with Shuffle

def new_Features_pkl(start_partition, end_partition, data_dir, first_name, last_name, data, labels):

    number_of_new_features = 9
    num_attributes = 24
    num_timestamps = 60
    
    for i in range(start_partition-1,end_partition):
        
        each_partition = np.zeros((data[i].shape[0], num_timestamps*(num_attributes)))
        each_partition = np.array(data[i])
        new_partition = np.zeros((each_partition.shape[0], number_of_new_features*num_attributes))
        
        with tqdm(each_partition.shape[0]) as pbar:
            for j in range(0,each_partition.shape[0]):
                new_column = np.zeros((num_timestamps*num_attributes)) 
                new_column = each_partition[j,:]
                

                new_features = np.zeros(number_of_new_features*(num_attributes))

                for m in range(0,num_attributes):
                    
                    mean = np.mean(new_column[m*num_timestamps:(m+1)*num_timestamps])
                    new_features[((m)*number_of_new_features) + 0] = mean
                    median = np.median(new_column[m*num_timestamps:(m+1)*num_timestamps])
                    new_features[((m)*number_of_new_features) + 1] = median
                    std = np.std(new_column[m*num_timestamps:(m+1)*num_timestamps])
                    new_features[((m)*number_of_new_features) + 2] = std
                    
                    skewness = skew(new_column[m*num_timestamps:(m+1)*num_timestamps])
                    if skewness == np.nan:
                        skewness = new_partition[j-1, ((m)*number_of_new_features) + 3]
                    new_features[((m)*number_of_new_features) + 3] = skewness
                    
                    kurtosis_value = kurtosis(new_column[m*num_timestamps:(m+1)*num_timestamps])
                    if kurtosis_value == np.nan:
                        kurtosis_value = new_partition[j-1, ((m)*number_of_new_features) + 4]
                    new_features[((m)*number_of_new_features) + 4] = kurtosis_value
                    
                    indices = np.arange(num_timestamps)
                    weight_array = indices / num_timestamps
                    weighted_avg = np.average(new_column[m*num_timestamps:(m+1)*num_timestamps], weights=weight_array)
                    if weighted_avg == np.nan:
                        weighted_avg = new_partition[j-1, ((m)*number_of_new_features) + 5]
                    new_features[((m)*number_of_new_features) + 5] = weighted_avg
                    
                    last_value = new_column[(m+1)*num_timestamps-1]
                    new_features[((m)*number_of_new_features) + 6] = last_value
                    first_value = new_column[m*num_timestamps]
                    new_features[((m)*number_of_new_features) + 7] = first_value
                    
                    numerator = np.sum((new_column[m*num_timestamps:(m+1)*num_timestamps] - mean) * (indices - np.mean(indices)))
                    denominator = np.sum((new_column[m*num_timestamps:(m+1)*num_timestamps] - mean) ** 2)
                    slope = numerator / denominator
                    if slope == np.nan:
                        slope = new_partition[j-1, ((m)*number_of_new_features) + 8]
                    new_features[((m)*number_of_new_features) + 8] = slope
                
                    
                new_partition[j,:] = new_features
                
                pbar.update(1)
                

        print("P"+str(i+1)+" Nan-Value: "+ str(np.isnan(new_partition).any()))  
        X_train = new_partition
        Y_train = labels[i]


        num_samples = X_train.shape[0]
        shuffle_indices = np.random.permutation(num_samples)

        X_train_shuffled = X_train[shuffle_indices]
        Y_train_shuffled = Y_train[shuffle_indices]
    

        with open(data_dir + first_name + "Partition" + str(i+1) 
                       + last_name + ".pkl", 'wb') as f:
            pickle.dump(X_train_shuffled, f)

        with open(data_dir + first_name + "Partition" + str(i+1) 
                       + "_Labels" + last_name + ".pkl", 'wb') as f:
            pickle.dump(Y_train_shuffled, f)

### Reading the Data built by the Previous Notebooks (RUS_Tomek_Smote )

In [2]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/6_1_OUSampling_Concatenation_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "RUS_Tomek_Smote_" +"Partition" + str(i+1) + "_OUSampling_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "RUS_Tomek_Smote_" + "Partition" + str(i+1) + "_Labels_OUSampling_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [3]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_1_OUSampling_NewFeatures_KnnImputation/"
name_first_part = 'RUS_Tomek_Smote_'
name_last_part = '_OUSampling_WithoutC_NewFeatures_KnnImputation'


new_Features_pkl(1, 5, data_dir, name_first_part, name_last_part, X_train_array, Y_train_array)

19996it [02:12, 151.00it/s]


P1 Nan-Value: False


19998it [02:10, 152.93it/s]


P2 Nan-Value: False


19994it [02:10, 153.09it/s]


P3 Nan-Value: False


19996it [02:12, 150.64it/s]


P4 Nan-Value: False


19998it [02:10, 152.80it/s]

P5 Nan-Value: False


### Reading the Data built by the Previous Notebooks (RUS_Tomek_Adasyn)

In [4]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/6_1_OUSampling_Concatenation_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "RUS_Tomek_Adasyn_" +"Partition" + str(i+1) + "_OUSampling_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "RUS_Tomek_Adasyn_" + "Partition" + str(i+1) + "_Labels_OUSampling_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [5]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_1_OUSampling_NewFeatures_KnnImputation/"
name_first_part = 'RUS_Tomek_Adasyn_'
name_last_part = '_OUSampling_WithoutC_NewFeatures_KnnImputation'


new_Features_pkl(1, 5, data_dir, name_first_part, name_last_part, X_train_array, Y_train_array)

20061it [02:11, 152.91it/s]


P1 Nan-Value: False


20043it [02:11, 152.67it/s]


P2 Nan-Value: False


19955it [02:10, 153.41it/s]


P3 Nan-Value: False


19981it [02:11, 152.45it/s]


P4 Nan-Value: False


20044it [02:11, 152.34it/s]


P5 Nan-Value: False


### Reading the Data built by the Previous Notebooks (Smote)

In [7]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/6_2_OSampling_Concatenation_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "Smote_" +"Partition" + str(i+1) + "_OSampling_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "Smote_" + "Partition" + str(i+1) + "_Labels_OSampling_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [8]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_2_OSampling_NewFeatures_KnnImputation/"
name_first_part = 'Smote_'
name_last_part = '_OSampling_WithoutC_NewFeatures_KnnImputation'


new_Features_pkl(1, 5, data_dir, name_first_part, name_last_part, X_train_array, Y_train_array)

131644it [14:46, 148.43it/s]


P1 Nan-Value: False


156692it [17:11, 151.95it/s]


P2 Nan-Value: False


70894it [07:49, 151.02it/s]


P3 Nan-Value: False


88280it [09:48, 150.11it/s]


P4 Nan-Value: False


137224it [15:01, 152.13it/s]


P5 Nan-Value: False


### Reading the Data built by the Previous Notebooks (Adasyn)

In [9]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/6_2_OSampling_Concatenation_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "Adasyn_" +"Partition" + str(i+1) + "_OSampling_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "Adasyn_" + "Partition" + str(i+1) + "_Labels_OSampling_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [10]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_2_OSampling_NewFeatures_KnnImputation/"
name_first_part = 'Adasyn_'
name_last_part = '_OSampling_WithoutC_NewFeatures_KnnImputation'


new_Features_pkl(1, 5, data_dir, name_first_part, name_last_part, X_train_array, Y_train_array)

131540it [14:25, 152.00it/s]


P1 Nan-Value: False


156622it [17:39, 147.80it/s]


P2 Nan-Value: False


70907it [07:53, 149.73it/s]


P3 Nan-Value: False


88334it [10:03, 146.45it/s]


P4 Nan-Value: False


137253it [14:51, 153.89it/s]


P5 Nan-Value: False


### Reading the Data built by the Previous Notebooks (GNI)

In [11]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/6_2_OSampling_Concatenation_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "GaussianNoise_" +"Partition" + str(i+1) + "_OSampling_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "GaussianNoise_" + "Partition" + str(i+1) + "_Labels_OSampling_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [12]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_2_OSampling_NewFeatures_KnnImputation/"
name_first_part = 'GaussianNoise_'
name_last_part = '_OSampling_WithoutC_NewFeatures_KnnImputation'


new_Features_pkl(1, 5, data_dir, name_first_part, name_last_part, X_train_array, Y_train_array)

131644it [14:12, 154.40it/s]


P1 Nan-Value: False


156692it [17:05, 152.81it/s]


P2 Nan-Value: False


70894it [07:44, 152.71it/s]


P3 Nan-Value: False


88280it [09:36, 153.14it/s]


P4 Nan-Value: False


137224it [14:59, 152.48it/s]


P5 Nan-Value: False


### Reading the Data built by the Previous Notebooks (GNI, Tomek, RUS)

In [13]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/6_1_OUSampling_Concatenation_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "RUS_Tomek_GNI_" +"Partition" + str(i+1) + "_OUSampling_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "RUS_Tomek_GNI_" + "Partition" + str(i+1) + "_Labels_OUSampling_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [14]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_1_OUSampling_NewFeatures_KnnImputation/"
name_first_part = 'RUS_Tomek_GNI_'
name_last_part = '_OUSampling_WithoutC_NewFeatures_KnnImputation'


new_Features_pkl(1, 5, data_dir, name_first_part, name_last_part, X_train_array, Y_train_array)

19996it [02:10, 153.39it/s]


P1 Nan-Value: False


19998it [02:10, 153.36it/s]


P2 Nan-Value: False


19998it [02:10, 153.34it/s]


P3 Nan-Value: False


20000it [02:10, 153.38it/s]


P4 Nan-Value: False


19994it [02:10, 152.99it/s]

P5 Nan-Value: False


### Reading the Data built by the Previous Notebooks (TimeGAN)

In [5]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/6_2_OSampling_Concatenation_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "TimeGAN_" +"Partition" + str(i+1) + "_OSampling_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "TimeGAN_" + "Partition" + str(i+1) + "_Labels_OSampling_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [6]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_2_OSampling_NewFeatures_KnnImputation/"
name_first_part = 'TimeGAN_'
name_last_part = '_OSampling_WithoutC_NewFeatures_KnnImputation'


new_Features_pkl(1, 5, data_dir, name_first_part, name_last_part, X_train_array, Y_train_array)

131030it [13:41, 159.44it/s]


P1 Nan-Value: False


155401it [16:08, 160.41it/s]


P2 Nan-Value: False


69623it [07:14, 160.14it/s]


P3 Nan-Value: False


87245it [09:03, 160.57it/s]


P4 Nan-Value: False


136922it [14:13, 160.48it/s]


P5 Nan-Value: False


### Reading the Data built by the Previous Notebooks (TimeGAN, Tomek, RUS)

In [2]:
import pickle
import pandas as pd

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/6_1_OUSampling_Concatenation_KnnImputation/"

X_train_array = []
Y_train_array = []

num_partitions = 5
for i in range(0, num_partitions):
    with open(data_dir + "RUS_Tomek_TimeGAN_" +"Partition" + str(i+1) + "_OUSampling_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        X_train_array.append(pickle.load(f))
    with open(data_dir + "RUS_Tomek_TimeGAN_" + "Partition" + str(i+1) + "_Labels_OUSampling_WithoutC_Concatenation_KnnImputation" +".pkl", 'rb') as f:
        Y_train_array.append(pickle.load(f))

In [3]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

data_dir = "/Users/samskanderi/Documents/Research_Project/SWANSF/code/7_1_OUSampling_NewFeatures_KnnImputation/"
name_first_part = 'RUS_Tomek_TimeGAN_'
name_last_part = '_OUSampling_WithoutC_NewFeatures_KnnImputation'


new_Features_pkl(1, 5, data_dir, name_first_part, name_last_part, X_train_array, Y_train_array)

18773it [02:01, 154.85it/s]


P1 Nan-Value: False


19807it [02:02, 161.70it/s]


P2 Nan-Value: False


19965it [02:03, 161.50it/s]


P3 Nan-Value: False


19320it [01:59, 162.19it/s]


P4 Nan-Value: False


19899it [02:02, 162.05it/s]

P5 Nan-Value: False
